In [1]:
%matplotlib inline
import os, sys

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import f1_score, classification_report, confusion_matrix, make_scorer
from sklearn.preprocessing import OneHotEncoder
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
sys.path.append('/home/admin/github/wordroid.sblo.jp/lib/keras_ex')
from gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
#from make_model import MyKerasClassifier

In [3]:
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
Y = iris.target
N = Y.size
Y2 = keras.utils.to_categorical(Y, num_classes=3)

index = np.arange(N)
xtrain = X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0]]
xtest = X[index[index % 2 == 0],:]
yans = Y2[index[index % 2 == 0]]

In [139]:
# def RBFClassifier(
    
# ):
#     return RBFClassifier0(make_model)
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop, Adam


class RBFClassifier(KerasClassifier):
    
    def __init__(self, **sk_params):
        super().__init__(make_model, **sk_params)
    
    def fit(self, x, y, sample_weight=None, **kwargs):
        nn = x.shape[1]
        ### gamma
        if self.sk_params.get('gamma', 'scale') == 'scale':
            gamma = 1 / (nn * x.var())
        else:
            gamma = self.sk_params['gamma']
        ### activation
        if self.sk_params.get('activation', 'softmax') == 'softmax':
            activation = 'softmax'
        else:
            activation = self.sk_params['activation']
        ### callbacks
        if self.sk_params.get('callbacks', None) is None:
            early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.000, restore_best_weights=True)
            callbacks = [early_stopping]
        else:
            callbacks = self.sk_params['callbacks']
        
        num_cls = y.shape[1]
        self.set_params(
            nn=nn,
            num_cls=num_cls,
            gamma=gamma,
            activation=activation,
            callbacks=callbacks
        )
        return super().fit(x, y, **kwargs)
    
#     def predict(self, x, **kwargs):
#         pred = self.model.predict(x)
#         y2_pred = np.argmax(pred, 1)
#         return self.classes_[y2_pred]
    
#     def predict_proba(self, x, **kwargs):
#         probs = self.model.predict(x)
        
#         # check if binary classification
#         if probs.shape[1] == 1:
#             # first column is probability of class 0 and second is of class 1
#             probs = np.hstack([1 - probs, probs])
#         return probs

class RBFRegressor(KerasRegressor):
    
    def __init__(self, **sk_params):
        super().__init__(make_model, **sk_params)
    
    def fit(self, x, y, sample_weight=None, **kwargs):
        nn = x.shape[1]
        ### gamma
        if self.sk_params.get('gamma', 'scale') == 'scale':
            gamma = 1 / (nn * x.var())
        else:
            gamma = self.sk_params['gamma']
        ### activation
        if self.sk_params.get('activation', 'linear') == 'linear':
            activation = 'linear'
        else:
            activation = self.sk_params['activation']
        ### loss
        if self.sk_params.get('loss', 'mse') == 'mse':
            loss = 'mse'
        else:
            loss = self.sk_params['loss']
        ### callbacks
        if self.sk_params.get('callbacks', None) is None:
            early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.000, restore_best_weights=True)
            callbacks = [early_stopping]
        else:
            callbacks = self.sk_params['callbacks']
        
        num_cls = 1
        self.set_params(
            nn=nn,
            num_cls=num_cls,
            gamma=gamma,
            activation=activation,
            loss=loss,
            callbacks=callbacks
        )
        return super().fit(x, y, **kwargs)


def make_model_gkernel1(nn=3, num_lm=2, random_state=0, init_wgt=None):
    inp = Input(shape=(nn,), name='inp')
    oup = inp
    
    if init_wgt is None:
        np.random.seed(random_state)
        init_wgt = np.random.random_sample((num_lm, nn)) * 2 - 1
    #weights2 = [np.log(np.array([1/(2*nn*0.1)]))]
    weights2 = [np.log(np.array([1/(2*np.sqrt(nn/6)*2)]))]
    oup = GaussianKernel2(init_wgt, weights=weights2, name='gkernel')(oup)
#     oup = Dense(NUM_LM, activation='elu')(oup)
    
    model = Model(inp, oup, name='model_gkernel')
    return init_wgt, model

def make_model_gkernel2(
    nn=4, num_lm=2,
    random_state=0, lm=None, gamma=None):
    
    inp = Input(shape=(nn,), name='inp')
    oup = inp
    
    if lm is None:
        rs = np.random.RandomState(random_state)
        lm = rs.random_sample((num_lm, nn))
    if gamma is None:
        gamma = 1/(2*np.sqrt(nn/6)*2)
    weights = [np.log(np.array([gamma]))]
    oup = GaussianKernel2(lm, weights=weights, name='gkernel')(oup)
    
    model = Model(inp, oup, name='model_gkernel')
    return lm, model

def make_model_gkernel3(
    nn=4, num_lm=2,
    random_state=0, lm=None, gamma=None):

    inp = Input(shape=(nn,), name='inp')
    oup = inp
    
    if lm is None:
        rs = np.random.RandomState(random_state)
        lm = rs.random_sample((num_lm, nn))
    if gamma is None:
        gamma = 1/(2*np.sqrt(nn/6)*2)
    weights = [lm, np.log(np.array([gamma]))]
    oup = GaussianKernel3(num_landmark=num_lm, num_feature=nn, weights=weights, name='gkernel')(oup)
    
    model = Model(inp, oup, name='model_gkernel')
    return lm, model

def make_model_out(
    num_lm=2, num_cls=3,
    activation='softmax',
    reg_l1=0.0, reg_l2=0.0
):
    inp = Input(shape=(num_lm,), name='inp')
    oup = Dense(num_cls,
                activation=activation,
                kernel_regularizer=regularizers.l1_l2(reg_l1, reg_l2),
                bias_regularizer=regularizers.l1_l2(reg_l1, reg_l2)
               )(inp)
    
    model = Model(inp, oup, name='model_out')
    return model

def make_model(
    make_model_gkernel=make_model_gkernel2,
    make_model_out=make_model_out,
    reg_l1=0.0, reg_l2=0.0,
    nn=2,
    num_lm=2, lm=None, gamma=None,
    random_state=0,
    num_cls=2, activation='softmax',
    opt=0.05,
    loss='categorical_crossentropy'
):
    K.clear_session()
    
    lm, model_gkernel = make_model_gkernel(
        nn=nn, num_lm=num_lm,
        random_state=random_state, lm=lm, gamma=gamma
    )
    model_out = make_model_out(
        num_lm=num_lm, num_cls=num_cls,
        activation=activation,
        reg_l1=reg_l1, reg_l2=reg_l2
    )
    model_gkernel.summary()
    model_out.summary()
    
    inp = model_gkernel.inputs[0]
    oup = model_gkernel(inp)
    oup = model_out(oup)
    
    model = Model(inp, oup)
    model.summary()
    if isinstance(opt, float):
        opt = Adam(opt)
    model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
    return model

In [140]:
lm, model_gkernel = make_model_gkernel2()
print(model_gkernel.inputs[0])
model_gkernel.summary()

Tensor("inp_2:0", shape=(?, 4), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 4)                 0         
_________________________________________________________________
gkernel (GaussianKernel2)    (None, 2)                 1         
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________


In [141]:
model_out = make_model_out()
model_out.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [142]:
model = make_model(nn=4, num_cls=3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 4)                 0         
_________________________________________________________________
gkernel (GaussianKernel2)    (None, 2)                 1         
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)          

In [143]:
model.fit(xtrain, ytrain, epochs=5, verbose=2)

Epoch 1/5
 - 0s - loss: 1.1020 - acc: 0.1867
Epoch 2/5
 - 0s - loss: 1.0989 - acc: 0.3333
Epoch 3/5
 - 0s - loss: 1.1007 - acc: 0.3333
Epoch 4/5
 - 0s - loss: 1.0992 - acc: 0.3333
Epoch 5/5
 - 0s - loss: 1.0971 - acc: 0.3333


In [144]:
pred = model.predict(xtrain)
pred[:5]

array([[0.3179267 , 0.3625287 , 0.31954452],
       [0.32236826, 0.3587636 , 0.3188681 ],
       [0.30777726, 0.37123024, 0.3209926 ],
       [0.3130137 , 0.36672813, 0.32025814],
       [0.31645915, 0.36378855, 0.31975234]], dtype=float32)

In [145]:
np.random.choice(np.arange(xtrain.shape[0]), 3)

array([35, 20, 21])

In [174]:
# clf = RBFClassifier(num_lm=3)
# clf = RBFClassifier(
#     num_lm=3,
#     lm=xtrain[:3]
# )
# clf = RBFClassifier(
#     num_lm=50,
#     lm=xtrain[:50],
#     make_model_gkernel=make_model_gkernel3
# )
# early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.000, restore_best_weights=True)
# clf = RBFClassifier(
#     num_lm=3,
#     lm=xtrain[np.random.choice(np.arange(xtrain.shape[0]), 3)],
# )
# clf = RBFClassifier(
#     num_lm=10,
#     lm=xtrain[:10],
#     callbacks=[early_stopping],
#     make_model_gkernel=make_model_gkernel3
# )
# clf = RBFClassifier(
#     num_lm=3,
#     lm=xtrain[:3],
#     make_model_gkernel=make_model_gkernel3,
#     activation='sigmoid',
#     loss='binary_crossentropy'
# )
clf = RBFClassifier(
    num_lm=3,
    lm=xtrain[np.random.choice(np.arange(xtrain.shape[0]), 3)],
    reg_l1=0.08
)
clf

In [175]:
%%time
clf.fit(xtrain, ytrain, epochs=5000, batch_size=256, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 4)                 0         
_________________________________________________________________
gkernel (GaussianKernel2)    (None, 3)                 1         
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)        

In [173]:
clf.sk_params

{'activation': 'softmax',
 'callbacks': [<keras.callbacks.EarlyStopping at 0x7f0481576f28>],
 'gamma': 0.06392857199469391,
 'lm': array([[5.8, 2.7, 4.1, 1. ],
        [7.2, 3. , 5.8, 1.6],
        [5.4, 3.4, 1.5, 0.4]], dtype=float32),
 'nn': 4,
 'num_cls': 3,
 'num_lm': 3,
 'reg_l1': 0.05}

In [149]:
clf.filter_sk_params(make_model)

{'activation': 'softmax',
 'gamma': 0.06392857199469391,
 'lm': array([[5.1, 3.7, 1.5, 0.4],
        [7.2, 3.6, 6.1, 2.5],
        [5. , 3.3, 1.4, 0.2]], dtype=float32),
 'nn': 4,
 'num_cls': 3,
 'num_lm': 3}

In [150]:
clf.__dict__

{'build_fn': <function __main__.make_model>,
 'classes_': array([0, 1, 2]),
 'model': <keras.engine.training.Model at 0x7f048956cb00>,
 'n_classes_': 3,
 'sk_params': {'activation': 'softmax',
  'callbacks': [<keras.callbacks.EarlyStopping at 0x7f048a607198>],
  'gamma': 0.06392857199469391,
  'lm': array([[5.1, 3.7, 1.5, 0.4],
         [7.2, 3.6, 6.1, 2.5],
         [5. , 3.3, 1.4, 0.2]], dtype=float32),
  'nn': 4,
  'num_cls': 3,
  'num_lm': 3}}

In [151]:
clf.predict(xtrain)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2])

In [152]:
clf.predict_proba(xtest)[:5]

array([[9.9748862e-01, 2.5113549e-03, 1.5167067e-15],
       [9.9699736e-01, 3.0027183e-03, 4.2168548e-15],
       [9.9743623e-01, 2.5637774e-03, 1.5508172e-15],
       [9.9684578e-01, 3.1541726e-03, 3.7681895e-15],
       [9.9330914e-01, 6.6908351e-03, 5.7260328e-14]], dtype=float32)

In [153]:
clf.score(xtrain, ytrain)

75/75 [==============================] - 0s 1ms/step


0.9733333333333334

In [154]:
clf.score(xtest, yans)

75/75 [==============================] - 0s 43us/step


0.9066666706403097

In [155]:
est = RBFRegressor(
    num_lm=3,
    lm=xtrain[np.random.choice(np.arange(xtrain.shape[0]), 3)],
    make_model_gkernel=make_model_gkernel3
)
# est = RBFRegressor(
#     num_lm=10,
#     lm=xtrain[:10],
#     make_model_gkernel=make_model_gkernel3,
#     loss='mae'
# )

In [156]:
%%time
est.fit(xtrain, np.argmax(ytrain, axis=1), epochs=5000, batch_size=256, verbose=0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 4)                 0         
_________________________________________________________________
gkernel (GaussianKernel3)    (None, 3)                 13        
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)        

In [157]:
est.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 4)                 0         
_________________________________________________________________
model_gkernel (Model)        (None, 3)                 13        
_________________________________________________________________
model_out (Model)            (None, 1)                 4         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


In [158]:
est.predict(xtrain)

array([ 4.1015789e-02,  4.5636371e-02, -6.1730340e-02, -1.5157416e-02,
        6.3845068e-03, -7.8132600e-03,  6.3599020e-02, -6.8096444e-02,
       -7.6221079e-03, -3.8145751e-02, -1.4981389e-02,  4.3793812e-02,
        2.8158829e-02, -3.6716923e-02,  2.2999361e-02, -3.2580644e-03,
       -7.2024509e-02,  2.0808533e-02, -3.3044130e-02, -2.1510810e-02,
        1.7615731e-01,  4.6843663e-02,  6.6694632e-02,  3.6528692e-02,
        9.7051263e-04,  1.0942169e+00,  1.2627946e+00,  1.1669037e+00,
        8.2217121e-01,  1.1424670e+00,  1.0976777e+00,  1.2633791e+00,
        9.6326977e-01,  7.8316903e-01,  9.3389297e-01,  9.1438729e-01,
        1.1224124e+00,  1.0122391e+00,  1.5418468e+00,  6.6225457e-01,
        8.4935009e-01,  1.6902976e+00,  1.1091480e+00,  1.2275176e+00,
        1.1631700e+00,  1.1753547e+00,  8.5892248e-01,  8.6697018e-01,
        9.8284751e-01,  1.0164098e+00,  1.9240260e+00,  1.9707254e+00,
        2.1137049e+00,  2.0558376e+00,  2.1426411e+00,  1.9520097e+00,
      

In [159]:
est.__dict__

{'build_fn': <function __main__.make_model>,
 'model': <keras.engine.training.Model at 0x7f0488175cc0>,
 'sk_params': {'activation': 'linear',
  'callbacks': [<keras.callbacks.EarlyStopping at 0x7f048f72bd68>],
  'gamma': 0.06392857199469391,
  'lm': array([[4.5, 2.3, 1.3, 0.3],
         [5.7, 3. , 4.2, 1.2],
         [6.1, 3. , 4.9, 1.8]], dtype=float32),
  'loss': 'mse',
  'make_model_gkernel': <function __main__.make_model_gkernel3>,
  'nn': 4,
  'num_cls': 1,
  'num_lm': 3}}

In [160]:
est.score(xtrain, np.argmax(ytrain, axis=1))

75/75 [==============================] - 0s 1ms/step


-0.03028330460190773

In [161]:
est.score(xtest, np.argmax(yans, axis=1))

75/75 [==============================] - 0s 40us/step


-0.03228535955150922